<center>

# **CUSTOMER TRANSACTIONS ANALYSIS**

</center>

<img src="Fintech Data Analysis.jpg" style="width:100%; height:400px; object-fit:cover;">

# Background

The fintech (financial technology) industry is rapidly transforming how individuals and businesses manage, transfer, and invest money. With the widespread adoption of mobile banking, digital payments, peer-to-peer lending, robo-advisory platforms, and blockchain-based services, fintech companies generate and collect massive volumes of data daily - from user transactions and loan applications to app interactions and fraud patterns.

# Problem Statement

The product team has requested a report on all successful transactions that have occurred within a time frame.

# Objective

- Analyze customer transactions to generate insights for the product and finance teams
- The report should include both savings and investment transactions, along with key metrics like cash flow, deposits, withdrawals, and breakage fees 

# Importing Libraries

In [1]:
## importing the necessary libraries

import pandas as pd

# 1. Data Loading and Inspection

In [2]:
## loading the CSV file that contains the SQL query
## displaying the first five rows

fintech = pd.read_csv("Fintech Data Analysis - Results from SQL Query.csv")
fintech.head()

,Customer ID,Customer Name,Sign-Up Date,KYC Completed Date,Investment Type,Sign-Up Cohort,KYC Cohort,Transaction ID,Transaction Type,USD Amount,Transaction Date,GHS Amount,Exchange Rate,Transaction Cohort,Week Number of Transaction,Asset Type,Maturity Date
0,NaN,,NaN,NaN,NaN,NaN,NaN,6f28ac6b-2cfc-4ec7-8a1a-a665a63cf678,deposit,1.17,2024-08-26 09:40:00.220233+00,20.0,17.08,2024-08-01 00:00:00+00,35,NaN,NaN
1,NaN,,NaN,NaN,NaN,NaN,NaN,bdf9951c-916b-4e66-96c4-aa72c025acfa,deposit,1.18,2024-10-10 09:40:00.234441+00,20.0,16.92,2024-10-01 00:00:00+00,41,NaN,NaN
2,NaN,,NaN,NaN,NaN,NaN,NaN,80e29a97-c2e9-4fa8-923d-c7cbb5394412,deposit,6.38,2025-03-07 10:26:14.436273+00,100.0,15.66,2025-03-01 00:00:00+00,10,NaN,NaN
3,NaN,,NaN,NaN,NaN,NaN,NaN,1b2a659c-ae60-4975-854d-8f4f40f36b54,deposit,1.18,2024-10-10 11:49:22.313753+00,20.0,16.92,2024-10-01 00:00:00+00,41,NaN,NaN
4,NaN,,NaN,NaN,NaN,NaN,NaN,971533a7-3c22-48c5-9a4e-a6312bd77835,deposit,6.41,2025-03-05 16:10:48.464645+00,100.0,15.60,2025-03-01 00:00:00+00,10,NaN,NaN


In [3]:
## displaying the shape of the dataset

fintech.shape

(149, 17)

In [4]:
## displaying basic info of the dataset

fintech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer ID                 4 non-null      object 
 1   Customer Name               149 non-null    object 
 2   Sign-Up Date                4 non-null      object 
 3   KYC Completed Date          4 non-null      object 
 4   Investment Type             4 non-null      object 
 5   Sign-Up Cohort              4 non-null      object 
 6   KYC Cohort                  4 non-null      object 
 7   Transaction ID              149 non-null    object 
 8   Transaction Type            149 non-null    object 
 9   USD Amount                  149 non-null    float64
 10  Transaction Date            149 non-null    object 
 11  GHS Amount                  149 non-null    float64
 12  Exchange Rate               149 non-null    float64
 13  Transaction Cohort          149 non

# 2. Data Cleaning

### 2.1 Handling Date Fields

In [5]:
## changing the datatypes of the date fields for the purpose of the analysis
 
fintech["Transaction Date"] = pd.to_datetime(fintech["Transaction Date"], errors = 'coerce').dt.tz_localize(None)
fintech["Maturity Date"] = pd.to_datetime(fintech["Maturity Date"], errors = 'coerce').dt.tz_localize(None)

In [6]:
## checking to see if there are null entries in Transaction Date

fintech["Transaction Date"].isnull().sum()

0

In [7]:
## checking to see if there are null entries in Maturity Date

fintech["Maturity Date"].isnull().sum()

145

There are null values in the Maturity Date field

In [8]:
## checking to see that the null entries in Maturity Date are pandas accepted NaTs

fintech["Maturity Date"].unique()

<DatetimeArray>
['NaT', '2025-05-06 00:00:00', '2025-06-15 00:00:00', '2025-09-26 00:00:00']
Length: 4, dtype: datetime64[ns]

# 3. Metrics Generation 

### 3.1 Breakage Fee

For withdrawals of certain asset types (e.g., "Risevest Real Estate" and "Risevest Fixed Income"), calculate a breakage fee of 2.5% if the withdrawal occurs before the maturity date. Duplicate rows for transactions where a breakage fee applies and update the transaction type and amounts accordingly

In [9]:
## checking the number and names of the non-null asset types

fintech["Asset Type"].value_counts() 

Asset Type
Risevest real estate     3
Risevest fixed income    1
Name: count, dtype: int64

In [10]:
## defining breakage fee asset types

breakage_assets = ["Risevest Real Estate", "Risevest Fixed Income"]

In [11]:
## identify the early withdrawals that qualify for the breakage fee
## the type of transaction is withdrawal
## the asset type in in the breakage_assets list
## transaction date for the withdrawal occurs before the maturity date

early_withdrawals = fintech[
    (fintech["Transaction Type"] == "withdrawal") &
    (fintech["Asset Type"].isin(breakage_assets)) &
    (fintech["Transaction Date"] < fintech["Maturity Date"])
].copy()

In [12]:
## applying the 2.5% breakage fee

early_withdrawals["USD Amount"] *= 0.025
early_withdrawals["GHS Amount"] *= 0.025
early_withdrawals["Transaction Type"] = "breakage_fee"

In [13]:
## displaying the first five rows of early_withdrawals

early_withdrawals.head()

,Customer ID,Customer Name,Sign-Up Date,KYC Completed Date,Investment Type,Sign-Up Cohort,KYC Cohort,Transaction ID,Transaction Type,USD Amount,Transaction Date,GHS Amount,Exchange Rate,Transaction Cohort,Week Number of Transaction,Asset Type,Maturity Date


Early_withdrawals is now a table

In [14]:
## appending the early withdrawals table to the fintech df

fintech = pd.concat([fintech, early_withdrawals], ignore_index=True)
fintech.head()

,Customer ID,Customer Name,Sign-Up Date,KYC Completed Date,Investment Type,Sign-Up Cohort,KYC Cohort,Transaction ID,Transaction Type,USD Amount,Transaction Date,GHS Amount,Exchange Rate,Transaction Cohort,Week Number of Transaction,Asset Type,Maturity Date
0,NaN,,NaN,NaN,NaN,NaN,NaN,6f28ac6b-2cfc-4ec7-8a1a-a665a63cf678,deposit,1.17,2024-08-26 09:40:00.220233,20.0,17.08,2024-08-01 00:00:00+00,35,NaN,NaT
1,NaN,,NaN,NaN,NaN,NaN,NaN,bdf9951c-916b-4e66-96c4-aa72c025acfa,deposit,1.18,2024-10-10 09:40:00.234441,20.0,16.92,2024-10-01 00:00:00+00,41,NaN,NaT
2,NaN,,NaN,NaN,NaN,NaN,NaN,80e29a97-c2e9-4fa8-923d-c7cbb5394412,deposit,6.38,2025-03-07 10:26:14.436273,100.0,15.66,2025-03-01 00:00:00+00,10,NaN,NaT
3,NaN,,NaN,NaN,NaN,NaN,NaN,1b2a659c-ae60-4975-854d-8f4f40f36b54,deposit,1.18,2024-10-10 11:49:22.313753,20.0,16.92,2024-10-01 00:00:00+00,41,NaN,NaT
4,NaN,,NaN,NaN,NaN,NaN,NaN,971533a7-3c22-48c5-9a4e-a6312bd77835,deposit,6.41,2025-03-05 16:10:48.464645,100.0,15.60,2025-03-01 00:00:00+00,10,NaN,NaT


In [15]:
## displaying the shape of the fintech df

fintech.shape

(149, 17)

### 3.2 Cash Flow Metrics

Cash Flow Metrics: Calculate the following for both USD and GHS:
- Cash Flow (usd_cash_flow, ghs_cash_flow): Positive for deposits, negative for withdrawals
- Deposits (for both usd and ghs deposits)
- Withdrawals (for both usd and ghs deposits)

### 3.21 Cash Flow (usd_cash_flow, ghs_cash_flow): Positive for deposits, negative for withdrawals

In [16]:
## computing the metrics for cash flow

def compute_cash_flow(row, currency):
    amount = row[f"{currency} Amount"]
    if row["Transaction Type"] == "deposit":
        return amount
    elif row["Transaction Type"] in ["withdrawal", "breakage_fee"]:
        return -amount
    return 0

fintech["usd_cash_flow"] = fintech.apply(lambda row: compute_cash_flow(row, "USD"), axis=1)
fintech["ghs_cash_flow"] = fintech.apply(lambda row: compute_cash_flow(row, "GHS"), axis=1)

In [17]:
## displaying the shape of the fintech df

fintech.shape

(149, 19)

### 3.22 Deposits (for both usd and ghs deposits)

In [18]:
## calculating the deposits in usd and ghs

fintech["usd_deposit"] = fintech.apply(lambda row: row["USD Amount"] if row["Transaction Type"] == "deposit" else 0, axis=1)
fintech["ghs_deposit"] = fintech.apply(lambda row: row["GHS Amount"] if row["Transaction Type"] == "deposit" else 0, axis=1)

In [19]:
## displaying the shape of the fintech df

fintech.shape

(149, 21)

### 3.23 Withdrawals (for both usd and ghs deposits)

In [20]:
## calculating the withdrawals in usd and ghs

fintech["usd_withdrawal"] = fintech.apply(lambda row: row["USD Amount"] if row["Transaction Type"] == "withdrawal" else 0, axis=1)
fintech["ghs_withdrawal"] = fintech.apply(lambda row: row["GHS Amount"] if row["Transaction Type"] == "withdrawal" else 0, axis=1)

In [21]:
## displaying the shape of the fintech df

fintech.shape

(149, 23)

### 3.3 Funding Source

Add a column funding_source:
- If the transaction type is internal_transfer, set the funding source to "flex dollar"
- Otherwise, set it to “mobile money”

In [22]:
## adding the column funding_source

fintech["funding_source"] = fintech["Transaction Type"].apply(
    lambda x: "flex dollar" if x == "internal_transfer" else "mobile money")

In [23]:
## displaying the shape of the fintech df

fintech.shape

(149, 24)

# 4. Saving The Final Processed File

In [24]:
## saving to a new CSV

fintech.to_csv("Fintech Data Analysis - Final Report.csv", index=False)